In [1]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install "shapely==2.0.5" trimesh pygltflib


In [2]:
from shapely.geometry import Polygon, box
import trimesh, pygltflib
print("OK:", Polygon, box)


OK: <class 'shapely.geometry.polygon.Polygon'> <function box at 0x00000143CE718400>


In [7]:
# ---- TEMP FIX: stub torchvision.ops.nms so Ultralytics runs without torchvision ----
import sys, types, torch

def _box_iou_one_to_many(b, B):
    # b: (4,) xyxy ; B: (N,4) xyxy  -> IoU (N,)
    x1 = torch.maximum(b[0], B[:, 0])
    y1 = torch.maximum(b[1], B[:, 1])
    x2 = torch.minimum(b[2], B[:, 2])
    y2 = torch.minimum(b[3], B[:, 3])
    inter = (x2 - x1).clamp(min=0) * (y2 - y1).clamp(min=0)
    area_b  = (b[2] - b[0]).clamp(min=0) * (b[3] - b[1]).clamp(min=0)
    area_B  = (B[:, 2] - B[:, 0]).clamp(min=0) * (B[:, 3] - B[:, 1]).clamp(min=0)
    return inter / (area_b + area_B - inter + 1e-6)

def _nms(boxes, scores, iou_threshold=0.45):
    """
    Minimal NMS replacement: returns indices of kept boxes (LongTensor).
    boxes:  (N,4) xyxy on any device
    scores: (N,)  confidence
    """
    if boxes.numel() == 0:
        return boxes.new_zeros((0,), dtype=torch.long)
    # sort by score desc
    idxs = scores.argsort(descending=True)
    keep = []
    while idxs.numel() > 0:
        i = idxs[0]
        keep.append(i)
        if idxs.numel() == 1:
            break
        ious = _box_iou_one_to_many(boxes[i], boxes[idxs[1:]])
        idxs = idxs[1:][ious <= iou_threshold]
    return torch.stack(keep) if keep else boxes.new_zeros((0,), dtype=torch.long)

# register stub module(s) so `import torchvision` and `from torchvision import ops` succeed
_tv = types.ModuleType("torchvision")
_ops = types.ModuleType("torchvision.ops")
_ops.nms = _nms
_tv.ops = _ops
sys.modules["torchvision"] = _tv
sys.modules["torchvision.ops"] = _ops
# -----------------------------------------------------------------------------


In [35]:
import os, cv2, numpy as np, torch
from pathlib import Path
from ultralytics import YOLO

# ================== CONFIG ==================
# --- REQUIRED CONFIG (put this ABOVE your "load models" lines) ---
from pathlib import Path
from ultralytics import YOLO

# paths you must set:
UNET_CKPT    = Path(r"C:\Users\win11\Desktop\Ebni\segmentation\runs_unet_gray_resdrop_noalb\unet_best.pth")
YOLO_WEIGHTS = Path(r"C:\Users\win11\Desktop\Ebni\object detection\yolov8s.pt")  # your YOLO model
IMAGES_DIR   = Path(r"C:\Users\win11\Desktop\Ebni\segmentation\datasetfinal\test\images")  # folder of test images

# model/image size used by your U-Net
IMG_SIZE     = 512

# quick sanity checks (will raise a clear error if a path is wrong)
assert UNET_CKPT.exists(),    f"UNET_CKPT not found: {UNET_CKPT}"
assert YOLO_WEIGHTS.exists(), f"YOLO_WEIGHTS not found: {YOLO_WEIGHTS}"
assert IMAGES_DIR.exists(),   f"IMAGES_DIR not found: {IMAGES_DIR}"

# Model architecture params (match your training)
IMG_SIZE     = 512      # your U-Net input size
IN_CHANNELS  = 1
BASE_CH      = 64
DROPOUT_P    = 0.20
BOTTLENECK_DROPOUT_P = 0.30

# Openings handling
DOOR_NAMES   = {"door", "doors"}     # will map YOLO names to these sets (lower-cased)
WINDOW_NAMES = {"window", "windows"}
OPENING_PAD  = 2                     # pixels to pad each bbox before cutting
MIN_BOX_AREA = 9                     # ignore tiny detections (in mask pixels)

# Output
OUT_DIR = Path("combined_outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# ===========================================


# -------- U-Net (same as in training) --------
import torch.nn as nn

class ResDoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch, p_drop=0.2, residual=True):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, padding=1, bias=False)
        self.bn1   = nn.BatchNorm2d(out_ch)
        self.act   = nn.ReLU(inplace=True)
        self.drop  = nn.Dropout2d(p_drop) if p_drop and p_drop>0 else nn.Identity()
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding=1, bias=False)
        self.bn2   = nn.BatchNorm2d(out_ch)
        self.skip  = nn.Conv2d(in_ch, out_ch, 1, bias=False) if in_ch != out_ch else nn.Identity()
        self.residual = residual
    def forward(self, x):
        identity = x
        out = self.act(self.bn1(self.conv1(x)))
        out = self.drop(out)
        out = self.bn2(self.conv2(out))
        if self.residual:
            identity = self.skip(identity)
            out = out + identity
        out = self.act(out)
        return out

class Down(nn.Module):
    def __init__(self, in_ch, out_ch, p_drop=0.2):
        super().__init__()
        self.pool = nn.MaxPool2d(2)
        self.block = ResDoubleConv(in_ch, out_ch, p_drop=p_drop)
    def forward(self, x): return self.block(self.pool(x))

class Up(nn.Module):
    def __init__(self, in_ch, out_ch, p_drop=0.2):
        super().__init__()
        self.up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=False)
        self.block = ResDoubleConv(in_ch, out_ch, p_drop=p_drop)
    def forward(self, x1, x2):
        x1 = self.up(x1)
        dy = x2.size(2) - x1.size(2); dx = x2.size(3) - x1.size(3)
        x1 = nn.functional.pad(x1, [dx//2, dx-dx//2, dy//2, dy-dy//2])
        return self.block(torch.cat([x2, x1], dim=1))

class UNetResDrop(nn.Module):
    def __init__(self, in_channels=1, num_classes=1, base_ch=64,
                 p_drop=0.2, p_drop_bot=0.3):
        super().__init__()
        self.inc   = ResDoubleConv(in_channels, base_ch, p_drop=p_drop)
        self.down1 = Down(base_ch,   base_ch*2, p_drop=p_drop)
        self.down2 = Down(base_ch*2, base_ch*4, p_drop=p_drop)
        self.down3 = Down(base_ch*4, base_ch*8, p_drop=p_drop)
        self.down4 = Down(base_ch*8, base_ch*16, p_drop=p_drop_bot)
        self.up1   = Up(base_ch*16+base_ch*8, base_ch*8,  p_drop=p_drop)
        self.up2   = Up(base_ch*8 +base_ch*4, base_ch*4,  p_drop=p_drop)
        self.up3   = Up(base_ch*4 +base_ch*2, base_ch*2,  p_drop=p_drop)
        self.up4   = Up(base_ch*2 +base_ch,   base_ch,    p_drop=p_drop)
        self.outc  = nn.Conv2d(base_ch, 1, 1)  # binary head
    def forward(self, x):
        x1 = self.inc(x); x2 = self.down1(x1); x3 = self.down2(x2); x4 = self.down3(x3); x5 = self.down4(x4)
        x  = self.up1(x5, x4); x = self.up2(x, x3); x = self.up3(x, x2); x = self.up4(x, x1)
        return self.outc(x)

def load_unet(ckpt_path: Path):
    ckpt = torch.load(str(ckpt_path), map_location="cpu")
    model = UNetResDrop(in_channels=IN_CHANNELS, num_classes=1,
                        base_ch=BASE_CH, p_drop=DROPOUT_P, p_drop_bot=BOTTLENECK_DROPOUT_P)
    state = ckpt["model"] if "model" in ckpt else ckpt
    model.load_state_dict(state, strict=True)
    model.to(DEVICE).eval()
    return model

def infer_wall_mask(model, img_path: Path, img_size=IMG_SIZE):
    # robust read (handle images that are already 1-channel or 3/4-channel)
    img = cv2.imread(str(img_path), cv2.IMREAD_UNCHANGED)
    if img is None:
        raise FileNotFoundError(img_path)

    # -> grayscale HxW (2D)
    if img.ndim == 2:
        img_gray = img
    elif img.ndim == 3:
        if img.shape[2] == 1:
            img_gray = img[:, :, 0]                    # squeeze singleton
        elif img.shape[2] == 3:
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        elif img.shape[2] == 4:
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY)
        else:
            raise ValueError(f"Unsupported channels {img.shape} for {img_path}")
    else:
        img_gray = np.squeeze(img)
        if img_gray.ndim != 2:
            raise ValueError(f"Could not get 2D gray from {img_path}, got {img_gray.shape}")

    H0, W0 = img_gray.shape[:2]
    # resize to model size (keep 2D)
    if (H0, W0) != (img_size, img_size):
        img_res = cv2.resize(img_gray, (img_size, img_size), interpolation=cv2.INTER_LINEAR)
    else:
        img_res = img_gray

    # normalize to [-1,1] -> add [C=1] -> add [B=1]  ==> [1,1,H,W]
    x = (img_res.astype(np.float32) / 255.0 - 0.5) / 0.5   # HxW
    x = torch.from_numpy(x).unsqueeze(0).unsqueeze(0).to(DEVICE)  # [1,1,H,W]
    # (optional) sanity check
    # print("x shape:", tuple(x.shape))  # should be (1, 1, img_size, img_size)

    model.eval()
    with torch.no_grad():
        logits = model(x)                      # [1,1,H,W] logits
        prob   = torch.sigmoid(logits)[0, 0].cpu().numpy()
        mask   = (prob > 0.5).astype(np.uint8) * 255

    return mask, (H0, W0)



# -------- YOLO helper (boxes -> opening mask) --------
def yolo_openings_mask(yolo_model, img_path: Path, mask_shape, names_to_ids=None):
    """
    Build a binary mask (same size as wall mask) where doors/windows boxes are filled (255).
    Rescales YOLO boxes from original image size -> mask size.
    """
    Hm, Wm = mask_shape
    img_bgr = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
    if img_bgr is None:
        raise FileNotFoundError(img_path)
    H0, W0 = img_bgr.shape[:2]
    sx, sy = Wm / W0, Hm / H0

    res = yolo_model(str(img_path))[0]

    # === Choose classes ===
    # Option A: by name (default)
    names = {i: n.lower() for i, n in getattr(yolo_model, "names", {}).items()}
    door_ids   = {i for i,n in names.items() if n in DOOR_NAMES}    # e.g., {"door"}
    window_ids = {i for i,n in names.items() if n in WINDOW_NAMES}  # e.g., {"window"}

    # Option B: force numeric IDs (uncomment and set if needed)
    # door_ids   = {0}
    # window_ids = {1}

    openings = np.zeros((Hm, Wm), np.uint8)
    kept = 0
    for b in res.boxes:
        cls = int(b.cls[0].item())
        if (cls not in door_ids) and (cls not in window_ids):
            continue
        x1, y1, x2, y2 = map(float, b.xyxy[0].tolist())
        X1 = int(max(0, x1 * sx - OPENING_PAD))
        Y1 = int(max(0, y1 * sy - OPENING_PAD))
        X2 = int(min(Wm, x2 * sx + OPENING_PAD))
        Y2 = int(min(Hm, y2 * sy + OPENING_PAD))
        if (X2 - X1) * (Y2 - Y1) < MIN_BOX_AREA:
            continue
        cv2.rectangle(openings, (X1, Y1), (X2, Y2), 255, thickness=-1)
        kept += 1

    # quick debug
    if kept == 0:
        print(f"[YOLO] {img_path.name}: no openings detected. names={names}")

    return openings


# ---------- Fail-safe 3D export: combined mask -> GLB with multi-pass fallback ----------
import numpy as np, cv2
from pathlib import Path

try:
    from shapely.geometry import Polygon, MultiPolygon
    from shapely.ops import unary_union
    import shapely.affinity as affinity
    from shapely.validation import make_valid
    _HAS_SHAPELY = True
except Exception as e:
    print("[ERROR] Shapely import failed:", e); _HAS_SHAPELY = False

try:
    import trimesh
    from trimesh.creation import extrude_polygon
    _HAS_TRIMESH = True
except Exception as e:
    print("[ERROR] Trimesh import failed:", e); _HAS_TRIMESH = False

# Tunables
PIXEL_TO_METER   = 0.01   # 1 px = 1 cm
WALL_HEIGHT_M    = 3.0
SIMPLIFY_EPS     = 0.0    # px; 0 disables
MIN_AREA_PX      = 4      # drop tiny specks
BASE_DILATE_PX   = 3      # default thickening
LINE_MODE_RATIO  = 0.02  # <0.3% => linework -> heavy thickening

def _polys_from_contours(mask_bw: np.ndarray, mode=cv2.RETR_CCOMP) -> MultiPolygon:
    """Contour→(multi)polygon with optional hole hierarchy when mode=RETR_CCOMP."""
    m = (mask_bw > 0).astype(np.uint8)
    # small cleanup
    m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8), 1)
    m = cv2.morphologyEx(m, cv2.MORPH_OPEN,  np.ones((3,3), np.uint8), 1)

    cnts, hier = cv2.findContours(m, mode, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts: 
        return MultiPolygon([])
    polys = []

    if mode == cv2.RETR_CCOMP and hier is not None:
        # parent/child hierarchy -> holes
        hier = hier[0]  # (N,4): [next, prev, first_child, parent]
        children = {}
        outer = []
        for i, h in enumerate(hier):
            parent = h[3]
            if parent == -1: outer.append(i)
            else: children.setdefault(parent, []).append(i)
        for oi in outer:
            ext = cnts[oi].squeeze(1)
            if ext.ndim != 2 or ext.shape[0] < 3: continue
            ext_poly = make_valid(Polygon(ext))
            if ext_poly.is_empty or ext_poly.area < MIN_AREA_PX: continue
            holes = []
            for hi in children.get(oi, []):
                hc = cnts[hi].squeeze(1)
                if hc.ndim != 2 or hc.shape[0] < 3: continue
                hole = make_valid(Polygon(hc))
                if hole.is_valid and not hole.is_empty and hole.area >= MIN_AREA_PX:
                    holes.append(hole.exterior.coords)
            poly = Polygon(ext_poly.exterior.coords, holes)
            if SIMPLIFY_EPS > 0: poly = poly.simplify(SIMPLIFY_EPS, preserve_topology=True)
            if poly.is_valid and not poly.is_empty: polys.append(poly)
    else:
        # externals only
        for c in cnts:
            ext = c.squeeze(1)
            if ext.ndim != 2 or ext.shape[0] < 3: continue
            poly = make_valid(Polygon(ext))
            if poly.is_empty or poly.area < MIN_AREA_PX: continue
            if SIMPLIFY_EPS > 0: poly = poly.simplify(SIMPLIFY_EPS, preserve_topology=True)
            if poly.is_valid and not poly.is_empty: polys.append(poly)

    if not polys: return MultiPolygon([])
    u = unary_union(polys)
    return MultiPolygon([u]) if isinstance(u, Polygon) else u

def _draw_poly_debug(mask_for_vec: np.ndarray, walls: MultiPolygon) -> np.ndarray:
    dbg = cv2.cvtColor((mask_for_vec>0).astype(np.uint8)*255, cv2.COLOR_GRAY2BGR)
    geoms = [walls] if isinstance(walls, Polygon) else list(walls.geoms)
    for poly in geoms:
        pts = np.array(poly.exterior.coords, np.int32).reshape(-1,1,2)
        cv2.polylines(dbg, [pts], True, (0,255,0), 2)
        for ring in poly.interiors:
            pts = np.array(ring.coords, np.int32).reshape(-1,1,2)
            cv2.polylines(dbg, [pts], True, (0,165,255), 2)
    return dbg

def export_glb_from_mask(combined_mask: np.ndarray, out_glb: Path,
                         pixel_to_meter: float = PIXEL_TO_METER,
                         wall_height_m: float = WALL_HEIGHT_M,
                         debug_png: Path | None = None) -> bool:
    """Multi-pass vectorize + extrude; writes debug overlay for each pass."""
    if not (_HAS_SHAPELY and _HAS_TRIMESH):
        print("[ERROR] Install shapely & trimesh in this kernel to export GLB.")
        return False

    H, W = combined_mask.shape[:2]
    m0 = combined_mask
    if m0.dtype != np.uint8: m0 = (m0>0).astype(np.uint8)*255

    nz = int((m0>0).sum()); ratio = nz/(H*W+1e-9)
    print(f"[GLB] nz={nz}  ratio={ratio:.4f}")
    if nz == 0:
        print("[WARN] empty combined mask; skipping GLB.")
        return False

    # Choose dilation
    if ratio < LINE_MODE_RATIO:
        ksz = max(3, int(round(min(H,W) * 0.02)))    # ~2% of min dimension
        print(f"[GLB] line-mode dilation: {ksz} px")
    else:
        ksz = BASE_DILATE_PX*2+1

    k = cv2.getStructuringElement(cv2.MORPH_RECT, (ksz, ksz))
    m1 = cv2.dilate(m0, k, 1)

    # Pass A: with holes (RETR_CCOMP)
    wallsA = _polys_from_contours(m1, mode=cv2.RETR_CCOMP)
    if not wallsA.is_empty:
        dbg = _draw_poly_debug(m1, wallsA) if debug_png else None
        return _extrude_and_save(wallsA, out_glb, pixel_to_meter, wall_height_m, dbg, debug_png)

    # Pass B: externals only
    wallsB = _polys_from_contours(m1, mode=cv2.RETR_EXTERNAL)
    if not wallsB.is_empty:
        dbg = _draw_poly_debug(m1, wallsB) if debug_png else None
        return _extrude_and_save(wallsB, out_glb, pixel_to_meter, wall_height_m, dbg, debug_png)

    # Pass C: invert and retry externals
    m2 = cv2.bitwise_not(m1)
    wallsC = _polys_from_contours(m2, mode=cv2.RETR_EXTERNAL)
    if not wallsC.is_empty:
        dbg = _draw_poly_debug(m2, wallsC) if debug_png else None
        return _extrude_and_save(wallsC, out_glb, pixel_to_meter, wall_height_m, dbg, debug_png)

    print("[WARN] All vectorization passes failed; GLB not written.")
    if debug_png:
        cv2.imwrite(str(debug_png), m1)  # at least write what we tried
    return False

def _extrude_and_save(walls: MultiPolygon, out_glb: Path,
                      px2m: float, height_m: float,
                      dbg_img: np.ndarray | None, dbg_path: Path | None) -> bool:
    geoms = [walls] if isinstance(walls, Polygon) else list(walls.geoms)
    print(f"[GLB] polygons to extrude: {len(geoms)}")
    meshes = []
    for p in geoms:
        try:
            p_m = affinity.scale(p, xfact=px2m, yfact=px2m, origin=(0,0)).buffer(0)
            meshes.append(extrude_polygon(p_m, height_m))
        except Exception as e:
            print("[WARN] skipping one polygon during extrusion:", e)
    if not meshes:
        print("[WARN] no meshes produced at extrusion step.")
        if dbg_img is not None and dbg_path is not None:
            cv2.imwrite(str(dbg_path), dbg_img)
        return False
    out_glb.parent.mkdir(parents=True, exist_ok=True)
    trimesh.Scene(meshes).export(str(out_glb))
    print(f"[OK] 3D walls saved -> {out_glb}")
    if dbg_img is not None and dbg_path is not None:
        cv2.imwrite(str(dbg_path), dbg_img)
    return True




# -------- Combine + visualize --------
def combine_and_save(unet_model, yolo_model, img_path: Path):
    # 1) U-Net mask
    wall_mask, (H0, W0) = infer_wall_mask(unet_model, img_path, IMG_SIZE)

    # 2) YOLO openings
    openings_mask = yolo_openings_mask(yolo_model, img_path, wall_mask.shape)

    # (optional) dilate openings so they fully cut walls
    if OPENING_PAD > 0:
        k = cv2.getStructuringElement(cv2.MORPH_RECT, (OPENING_PAD*2+1, OPENING_PAD*2+1))
        openings_mask = cv2.dilate(openings_mask, k, 1)

    # 3) subtract
    combined = cv2.bitwise_and(wall_mask, cv2.bitwise_not(openings_mask))

    # 4) save rasters + overlay
    stem = img_path.stem
    OUT_DIR.mkdir(parents=True, exist_ok=True)
    cv2.imwrite(str(OUT_DIR / f"{stem}_wall_mask.png"),      wall_mask)
    cv2.imwrite(str(OUT_DIR / f"{stem}_openings_mask.png"),  openings_mask)
    cv2.imwrite(str(OUT_DIR / f"{stem}_combined_mask.png"),  combined)

    vis = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
    if vis is not None:
        comb_up = cv2.resize(combined, (vis.shape[1], vis.shape[0]), interpolation=cv2.INTER_NEAREST)
        edges = cv2.Canny(comb_up, 50, 150) > 0
        vis[edges] = (0, 255, 0)
        # ✅ Correct signature: path, image
        cv2.imwrite(str(OUT_DIR / f"{stem}_overlay.png"), vis)

    # 5) GLB export (debug messages print here)
        out_glb  = OUT_DIR / f"{stem}_walls.glb"
        vec_dbg  = OUT_DIR / f"{stem}_vec_debug.png"   # will show outlines it extracted
        ok = export_glb_from_mask(combined, out_glb, debug_png=vec_dbg)


    print(f"[OK] {stem}: saved PNGs; GLB={ok}")
    return ok



    
import sys
!{sys.executable} -m pip install shapely==2.0.5 trimesh pygltflib





# -------------- Run on one or many images --------------
if __name__ == "__main__":
    # load models
    # put this BEFORE `yolo = YOLO(...)`
    from ultralytics.nn import autobackend

    def _skip_warmup(self, imgsz=(1, 3, 640, 640)):
    # warmup only speeds up the first inference; skipping is safe
        return None

    autobackend.AutoBackend.warmup = _skip_warmup

    unet = load_unet(UNET_CKPT)
    yolo = YOLO(str(YOLO_WEIGHTS))

    # (A) run on one random image (your original flow)
    # files = [p for p in IMAGES_DIR.iterdir() if p.suffix.lower() in {".png",".jpg",".jpeg",".bmp",".tif",".tiff"}]
    # if not files:
    #     raise FileNotFoundError(f"No images under {IMAGES_DIR}")
    # import random
    # img_path = random.choice(files)
    # print("Selected:", img_path.name)
    # combine_and_save(unet, yolo, img_path)

    #(B) run on all images in folder (uncomment if you want batch)
    for p in files:
        try:
            combine_and_save(unet, yolo, p)
            

        except Exception as e:
            print("[WARN] failed on", p.name, ":", e)


C:\Users\win11\AppData\Local\Temp\ipykernel_3164\2211949984.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(str(ckpt_path), map_location="cpu")



image 1/1 C:\Users\win11\Desktop\Ebni\segmentation\datasetfinal\test\images\IMG-20250721-WA0011_jpg.rf.9397955eb1a89b77e029aa655491416e.png: 640x640 (no detections), 122.0ms
Speed: 5.6ms preprocess, 122.0ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)


try running `pip install mapbox-earcut manifold3d`or `triangle`, `mapbox_earcut`, then explicitly pass:
`triangulate_polygon(*args, engine="triangle")`
to use the non-FSF-approved-license triangle engine


[YOLO] IMG-20250721-WA0011_jpg.rf.9397955eb1a89b77e029aa655491416e.png: no openings detected. names={0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toile

try running `pip install mapbox-earcut manifold3d`or `triangle`, `mapbox_earcut`, then explicitly pass:
`triangulate_polygon(*args, engine="triangle")`
to use the non-FSF-approved-license triangle engine
try running `pip install mapbox-earcut manifold3d`or `triangle`, `mapbox_earcut`, then explicitly pass:
`triangulate_polygon(*args, engine="triangle")`
to use the non-FSF-approved-license triangle engine
try running `pip install mapbox-earcut manifold3d`or `triangle`, `mapbox_earcut`, then explicitly pass:
`triangulate_polygon(*args, engine="triangle")`
to use the non-FSF-approved-license triangle engine
try running `pip install mapbox-earcut manifold3d`or `triangle`, `mapbox_earcut`, then explicitly pass:
`triangulate_polygon(*args, engine="triangle")`
to use the non-FSF-approved-license triangle engine
try running `pip install mapbox-earcut manifold3d`or `triangle`, `mapbox_earcut`, then explicitly pass:
`triangulate_polygon(*args, engine="triangle")`
to use the non-FSF-approved-lice

...

[GLB] polygons to extrude: 29
[WARN] skipping one polygon during extrusion: No available triangulation engine!
[WARN] skipping one polygon during extrusion: No available triangulation engine!
[WARN] skipping one polygon during extrusion: No available triangulation engine!
[WARN] skipping one polygon during extrusion: No available triangulation engine!
[WARN] skipping one polygon during extrusion: No available triangulation engine!
[WARN] skipping one polygon during extrusion: No available triangulation engine!
[WARN] skipping one polygon during extrusion: No available triangulation engine!
[WARN] skipping one polygon during extrusion: No available triangulation engine!
[WARN] skipping one polygon during extrusion: No available triangulation engine!
[WARN] skipping one polygon during extrusion: No available triangulation engine!
[WARN] skipping one polygon during extrusion: No available triangulation engine!
[WARN] skipping one polygon during extrusion: No available triangulation engine

try running `pip install mapbox-earcut manifold3d`or `triangle`, `mapbox_earcut`, then explicitly pass:
`triangulate_polygon(*args, engine="triangle")`
to use the non-FSF-approved-license triangle engine
try running `pip install mapbox-earcut manifold3d`or `triangle`, `mapbox_earcut`, then explicitly pass:
`triangulate_polygon(*args, engine="triangle")`
to use the non-FSF-approved-license triangle engine
try running `pip install mapbox-earcut manifold3d`or `triangle`, `mapbox_earcut`, then explicitly pass:
`triangulate_polygon(*args, engine="triangle")`
to use the non-FSF-approved-license triangle engine
try running `pip install mapbox-earcut manifold3d`or `triangle`, `mapbox_earcut`, then explicitly pass:
`triangulate_polygon(*args, engine="triangle")`
to use the non-FSF-approved-license triangle engine
try running `pip install mapbox-earcut manifold3d`or `triangle`, `mapbox_earcut`, then explicitly pass:
`triangulate_polygon(*args, engine="triangle")`
to use the non-FSF-approved-lice

[YOLO] w1024-3-_jpg.rf.75fb90e75925d6e4f438d0e9f0e2ce01.png: no openings detected. names={0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv

try running `pip install mapbox-earcut manifold3d`or `triangle`, `mapbox_earcut`, then explicitly pass:
`triangulate_polygon(*args, engine="triangle")`
to use the non-FSF-approved-license triangle engine
try running `pip install mapbox-earcut manifold3d`or `triangle`, `mapbox_earcut`, then explicitly pass:
`triangulate_polygon(*args, engine="triangle")`
to use the non-FSF-approved-license triangle engine


[GLB] nz=4190  ratio=0.0160
[GLB] line-mode dilation: 10 px
[GLB] polygons to extrude: 2
[WARN] skipping one polygon during extrusion: No available triangulation engine!
[WARN] skipping one polygon during extrusion: No available triangulation engine!
[WARN] no meshes produced at extrusion step.
[OK] w800x533-10-_jpg.rf.df9c96227515843965e7329dd677a0ac: saved PNGs; GLB=False
